In [2]:
#Import
from pymatgen.analysis.interfaces.substrate_analyzer import SubstrateAnalyzer
from pymatgen.analysis.interfaces.coherent_interfaces import CoherentInterfaceBuilder
from pymatgen.analysis.interfaces.zsl import ZSLGenerator
from pymatgen.core.structure import Structure

from pymatgen.core.surface import get_symmetrically_distinct_miller_indices
from pymatgen.core.surface import SlabGenerator

from pymatgen.ext.matproj import MPRester
api_key = "kJhjnOu7tx7q2ddENmIhMexGuOujnGcV"


import crystal_toolkit
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt


import os
from ase.optimize import FIRE
from ase.io import read, write
from ase.atoms import Atoms
from sevenn.calculator import SevenNetCalculator


# Conversion factor
ev_per_a2_to_j_per_m2 = 16.0217657

# Li anode coatings

### Get final list of suitable compounds

In [3]:
#paths to data tables with suitable coatings and barriers
path_table_coatings = 'data/Li_anode_coatings_relaxed.csv'
path_table_barriers = 'data/Li_percolation_barriers_MACE.csv'
path_table_sol = 'data/LLZO_dop_small2 - LLZO_dop_small2.csv'

table_coatings = pd.read_csv(path_table_coatings)
table_barriers = pd.read_csv(path_table_barriers)
table_sol = pd.read_csv(path_table_sol)

In [4]:
# set criteria for barrier limit
e_m_lim = 0.6 #eV
e_sol_lim = 0.510 #eV
concentration_lim = 0.0001 #eV

In [6]:
# get final list of compounds

list_of_compounds = []
list_of_compounds2 = []
list_of_compounds_final = []

for index, row in table_barriers.iterrows():
    e_m = round(row['e3d'],2)
    mat_id = row['material_id']
    if 0 < e_m < e_m_lim: 
        # print(f"Row {index}: Em 3d = {e_m}, material_id = {mat_id}")
        list_of_compounds.append(mat_id)
        
# for index, row in table_sol.iterrows():
#     if row['d_energy_dft'] != 'No':
#         e_sol = round(float(row['d_energy_dft']),3)
#         mat_id = row['name_maxr'].split('_')[1]
#         if e_sol < e_sol_lim: 
#             list_of_compounds2.append(mat_id)
            
            
for index, row in table_sol.iterrows():
    # if row['C_T300'] != 'No':
        conc = round(float(row['C_T300']),5)
        mat_id = row['material_id']#.split('_')[1]
        if conc > concentration_lim: 
            list_of_compounds2.append(mat_id)
            
            
            
list_of_compounds_sol_mig = (set(list_of_compounds)&set(list_of_compounds2))
        
for index, row in table_coatings.iterrows():
    mat_id = row['material_id']
    if mat_id in list_of_compounds_sol_mig:
        formula = row['formula_pretty']
        # print(f'Compound {formula}')
        list_of_compounds_final.append(mat_id)

# print(list_of_compounds,list_of_compounds2)
new_df = table_coatings[table_coatings["material_id"].isin(list_of_compounds_sol_mig)]
merged_df = pd.merge(new_df, table_barriers, on="material_id", how="inner")
df_sorted = merged_df.sort_values(by="e3d")
df_sorted.index = range(1, len(df_sorted) + 1)

# Reordering columns
df_sorted2 = df_sorted[[
    "formula_pretty", "space_group", "crystal_system", "nsites",
    "e1d", "e2d", "e3d", "fmax", "material_id", "theoretical",
    "reduction_limit", "oxidation_limit", "reduction_reaction",
    "oxidation_reaction", "chemsys", 
    "energy_above_hull", "band_gap", "is_stable"
]]

# Rounding e1d, e2d, and e3d to 2 decimal places
df_sorted2[["e1d", "e2d", "e3d"]] = df_sorted2[["e1d", "e2d", "e3d"]].round(2)

In [7]:
list_of_compounds_sol_mig, len(list_of_compounds_sol_mig)

({'mp-1189725',
  'mp-13944',
  'mp-23703',
  'mp-2530',
  'mp-3887',
  'mp-542435',
  'mp-569782',
  'mp-570935',
  'mp-755225'},
 9)

In [8]:
df_sorted2

,formula_pretty,space_group,crystal_system,nsites,e1d,e2d,e3d,fmax,material_id,theoretical,reduction_limit,oxidation_limit,reduction_reaction,oxidation_reaction,chemsys,energy_above_hull,band_gap,is_stable
1,Li3ScN2,Ia-3,Cubic,48,0.12,0.12,0.12,0.075864,mp-542435,False,0.00,0.59,8 Li3ScN2 -> 8 Li3ScN2,8 Li3ScN2 -> 8 ScN + 2.667 LiN3 + 21.33 Li,Li-N-Sc,0.0,2.24,True
2,Li2HN,Pnma,Orthorhombic,16,0.09,0.12,0.12,0.089242,mp-1189725,True,0.25,0.66,4 Li2HN -> 4 Li2HN,4 Li2HN -> 2 LiH2N + 0.6667 LiN3 + 5.333 Li,H-Li-N,0.0,2.08,True
3,Li3GaN2,Ia-3,Cubic,48,0.15,0.15,0.15,0.086796,mp-3887,False,0.13,0.77,8 Li3GaN2 -> 8 Li3GaN2,8 Li3GaN2 -> 8 GaN + 2.667 LiN3 + 21.33 Li,Ga-Li-N,0.0,2.38,True
4,Li3AlN2,Ia-3,Cubic,48,0.20,0.20,0.20,0.096252,mp-13944,False,0.00,0.79,8 Li3AlN2 -> 8 Li3AlN2,8 Li3AlN2 -> 8 AlN + 2.667 LiN3 + 21.33 Li,Al-Li-N,0.0,2.94,True
5,Sr2LiCBr3N2,Fd-3m,Cubic,36,0.28,0.28,0.28,0.078636,mp-569782,False,0.00,2.14,4 Sr2LiCBr3N2 -> 4 Sr2LiCBr3N2,4 Sr2LiCBr3N2 -> 2 SrCN2 + 6 SrBr2 + 2 N2 + 2 ...,Br-C-Li-N-Sr,0.0,3.97,True
6,LiI,P6_3mc,Hexagonal,4,0.46,0.46,0.46,0.077815,mp-570935,False,0.00,2.84,2 LiI -> 2 LiI,2 LiI -> 2 I + 2 Li,I-Li,0.0,4.38,True
7,LiH,Fm-3m,Cubic,2,0.47,0.47,0.47,0.072075,mp-23703,False,0.00,0.99,LiH -> LiH,LiH -> 0.5 H2 + Li,H-Li,0.0,2.98,True
8,Li2Te,Fm-3m,Cubic,3,0.49,0.49,0.49,0.076824,mp-2530,False,0.00,1.56,Li2Te -> Li2Te,Li2Te -> 0.3333 LiTe3 + 1.667 Li,Li-Te,0.0,2.49,True
9,Li8ZrO6,R-3,Trigonal,15,0.53,0.53,0.53,0.092343,mp-755225,True,0.05,2.88,Li8ZrO6 -> 2.5 Li2O + 0.5 Li6Zr2O7,Li8ZrO6 -> 0.5 Li6Zr2O7 + 1.25 Li2O2 + 2.5 Li,Li-O-Zr,0.0,4.55,False


In [13]:
list_of_compounds_final, len(list_of_compounds_final)

(['mp-755225',
  'mp-542435',
  'mp-3887',
  'mp-13944',
  'mp-23703',
  'mp-2530',
  'mp-1189725',
  'mp-570935',
  'mp-569782'],
 9)

### Get data from MP

In [15]:
import pymatgen.core as pmg
print(pmg.__version__)

2025.10.7


In [9]:
# Create a MPRester object with the API key
dic_st_final = {}
for matproj_id in list_of_compounds_final:

    with MPRester(api_key) as mpr:
        compounds = mpr.materials.summary.search(material_ids=[matproj_id],  
                                                 _fields = [ 'structure',
                                                            'material_id',
                                                            'symmetry',
                                                            'theoretical'
                                            ])

    # mpr = MPRester(api_key)
    # ws = mpr.get_wulff_shape("mp-985585")
    compound = compounds[0]
    st = compound['structure']
    # os.makedirs(f"interfaces_with_Li/{st.reduced_formula}/",exist_ok=True)
    dic_st_final[matproj_id] = st

In [10]:
dic_st_final

{'mp-755225': Structure Summary
 Lattice
     abc : 6.0182354003147776 6.018235924314327 6.018235959553977
  angles : 53.876177848863115 53.876172534879416 53.87617536814383
  volume : 132.07531174586885
       A : np.float64(2.726425899999999) np.float64(1.57410138) np.float64(5.12912897)
       B : np.float64(-2.72642585) np.float64(1.57410347) np.float64(5.12912897)
       C : np.float64(-1.05e-06) np.float64(-3.14820585) np.float64(5.12912897)
     pbc : True True True
 PeriodicSite: Li (-1.809, -0.1854, 7.006) [0.1039, 0.7675, 0.4946]
 PeriodicSite: Li (-1.065, 1.474, 8.381) [0.5054, 0.8961, 0.2325]
 PeriodicSite: Li (-0.7441, -1.66, 8.381) [0.2325, 0.5054, 0.8961]
 PeriodicSite: Li (-3.448e-07, -3.448e-07, 5.305) [0.3448, 0.3448, 0.3448]
 PeriodicSite: Li (-6.552e-07, -6.552e-07, 10.08) [0.6552, 0.6552, 0.6552]
 PeriodicSite: Li (0.7441, 1.66, 7.006) [0.7675, 0.4946, 0.1039]
 PeriodicSite: Li (1.065, -1.474, 7.006) [0.4946, 0.1039, 0.7675]
 PeriodicSite: Li (1.809, 0.1854, 8.381)

In [96]:
# st.sites[40].specie.oxi_state
# site.specie.symbol, 0) for site in surface_atoms

-3.0

# Create interfaces

In [16]:
import numpy as np
from pymatgen.core import Structure
from pymatgen.analysis.interfaces.zsl import ZSLGenerator
from pymatgen.analysis.interfaces.coherent_interfaces import CoherentInterfaceBuilder
from pymatgen.analysis.interfaces.substrate_analyzer import SubstrateAnalyzer


def matches(substrate_bulk, film_bulk, substrate_miller =None, film_max_miller =4, misfit = 5):
    # Find matches between fixed substrate and film with misfit criterion
    out_list = []
    # out_dic = {'substrate_hkl':None, 'film_hkl':None, 'misfit':None}
    
    sub_analyzer = SubstrateAnalyzer(film_max_miller =film_max_miller, bidirectional = 0)
    sub_analyzer.calculate(film=film_bulk,substrate=substrate_bulk)
    matches = list(sub_analyzer.calculate(film=film_bulk,substrate=substrate_bulk, substrate_millers=[substrate_miller]))
    # print(matches)

    filtered_matches = []
    film_millers = []
    # Process each match
    for match in matches:
        film_matrix = match.film_transformation
        substrate_matrix = match.substrate_transformation

        # Extract original in-plane lattice vectors from bulk film
        original_vectors_s = np.array([substrate_bulk.lattice.matrix[0], 
                                     substrate_bulk.lattice.matrix[1]])

        # Apply transformation matrix to get new film lattice vectors
        new_vectors_s = np.dot(substrate_matrix, original_vectors_s)

        # Compute misfit (strain) in x and y directions
        misfit_x_s = abs((np.linalg.norm(new_vectors_s[0]) - np.linalg.norm(original_vectors_s[0])) / np.linalg.norm(original_vectors_s[0]))
        misfit_y_s = abs((np.linalg.norm(new_vectors_s[1]) - np.linalg.norm(original_vectors_s[1])) / np.linalg.norm(original_vectors_s[1]))
        # Extract original in-plane lattice vectors from bulk film
        original_vectors = np.array([film_bulk.lattice.matrix[0], 
                                     film_bulk.lattice.matrix[1]])

        # Apply transformation matrix to get new film lattice vectors
        new_vectors = np.dot(film_matrix, original_vectors)

        # Compute misfit (strain) in x and y directions
        misfit_x = abs((np.linalg.norm(new_vectors[0]) - np.linalg.norm(original_vectors[0])) / np.linalg.norm(original_vectors[0]))
        misfit_y = abs((np.linalg.norm(new_vectors[1]) - np.linalg.norm(original_vectors[1])) / np.linalg.norm(original_vectors[1]))
        
        # print(f"Misfit: {misfit_x:.4f},{misfit_y:.4f}")
        print(f"Miller s/f: {substrate_miller}/{match.film_miller}",f"Misfit: {misfit_x:.2f},{misfit_y:.2f}")
        
        # Apply filtering conditions
        if misfit_x <= misfit and misfit_y <= misfit:
            filtered_matches.append(match)
            misfit_x = f'{misfit_x:.1f}'
            misfit_x_s = f'{misfit_x_s:.1f}'
            misfit_y = f'{misfit_y:.1f}'
            misfit_y_s = f'{misfit_y_s:.1f}'
            
            print('ok')
            
            if match.film_miller not in film_millers:
                film_millers.append(match.film_miller)
                
                out_list.append([substrate_miller, match.film_miller, [misfit_x, misfit_y], match.von_mises_strain, [misfit_x_s,misfit_y_s]])

                print(f"Miller s/f: {substrate_miller}/{match.film_miller}")
                print(f"Match area: {match.match_area:.4f}")
                print(f"Von_mises_strain: {match.von_mises_strain:.4f}")
                print(f"Film Misfit along x: {misfit_x}")
                print(f"Film Misfit along y: {misfit_y}")
                print(f"Sub Misfit along x: {misfit_x_s}")
                print(f"Sub Misfit along y: {misfit_y_s}\n\n")
                
    
    return(out_list)


def compute_surface_density(structure, select="top", layer_thickness=1.0):
    a_vector = structure.lattice.matrix[0]
    b_vector = structure.lattice.matrix[1]
    surface_area = np.linalg.norm(np.cross(a_vector, b_vector))
    cartesian_z_coords = np.array([site.coords[2] for site in structure])
    z_max = np.max(cartesian_z_coords)
    z_min = np.min(cartesian_z_coords)
    
    if select == "top":
        surface_atoms = [site for site in structure if (z_max - site.coords[2] <= layer_thickness)]
    elif select == "bottom":
        surface_atoms = [site for site in structure if (site.coords[2] - z_min <= layer_thickness)]
    else:
        raise ValueError("Invalid selection! Use 'top' or 'bottom'.")
    
    num_surface_atoms = len(surface_atoms)
    return num_surface_atoms / surface_area



def compute_surface_charge_density(structure, select="top", layer_thickness=1.0):
    
    # oxidation_states = {'O': -2, 'Li': +1, 'Cl': -1, 'F': -1, 'N' : -3, 'B': +3, 'I':  -1, 'Be':+2, 'Ga':+3,
    #                    'Sc': +3, 'Al': +3, 'H' : -1, 'S': -2, 'Br': -1, 'Te':-2, 'Tm': +3, 'Gd':+3, 'Sr':+2, 
                        # 'C': +4, 'Zr': +4, 'La': +3, 'P': +5, 'Si': +4}
    a_vector = structure.lattice.matrix[0]
    b_vector = structure.lattice.matrix[1]
    surface_area = np.linalg.norm(np.cross(a_vector, b_vector))
    cartesian_z_coords = np.array([site.coords[2] for site in structure])
    z_max = np.max(cartesian_z_coords)
    z_min = np.min(cartesian_z_coords)
    
    if select == "top":
        surface_atoms = [site for site in structure if (z_max - site.coords[2] <= layer_thickness)]
    elif select == "bottom":
        surface_atoms = [site for site in structure if (site.coords[2] - z_min <= layer_thickness)]
    else:
        raise ValueError("Invalid selection! Use 'top' or 'bottom'.")
    
    # surface_charge = sum(oxidation_states.get(site.specie.symbol, 0) for site in surface_atoms)
    # surface_charge = sum(oxidation_states.get(site.specie.symbol, 0) for site in surface_atoms)
     # Use oxidation state from site.specie
    try:
        surface_charge = sum(site.specie.oxi_state for site in surface_atoms)
    except AttributeError as e:
        raise ValueError("Structure must have oxidation states assigned to each site. You can use `Structure.add_oxidation_state_by_guess()` or provide them manually.") from e
    return surface_charge / surface_area

def create_interfaces(substrate_bulk, film_bulk, substrate_miller, film_miller, film_max_miller=4, num_sites_limit = 200, 
                      density_limit = 0.1, charge_limit = 0.1, gap=2.0, vacuum_over_film=15.0, film_thickness=5, substrate_thickness=7,
                     surface_thickness = 0.8, misfit = 5, match = None, folder_name = 'interfaces_with_Li'):
    i = -1
    all_interfaces = []
    dic_list = []
    sub_formula = substrate_bulk.composition.reduced_formula
    film_formula = film_bulk.composition.reduced_formula
    
    zsl = ZSLGenerator(max_area=400, max_area_ratio_tol=0.05, max_length_tol=0.05, max_angle_tol=1, bidirectional=False)
    seen_interfaces = set()
    
    cib = CoherentInterfaceBuilder(film_structure=film_bulk, substrate_structure=substrate_bulk, 
                                   film_miller=film_miller, substrate_miller=substrate_miller, 
                                   zslgen=zsl, filter_out_sym_slabs = True)
    
    terminations = cib.terminations
    print(terminations)

    for termination in terminations:
        interfaces = list(cib.get_interfaces(termination=termination, gap=gap, vacuum_over_film=vacuum_over_film,
                            film_thickness=film_thickness, substrate_thickness=substrate_thickness, in_layers=False))

        for interface in interfaces:
            # interface_id = (interface.num_sites, termination)
            # Check if substrate lattice was altered
            original_substrate_lattice = substrate_bulk.lattice.matrix
            new_substrate_lattice = interface.substrate.lattice.matrix

                   
                
            interface_id = (termination)
            if  interface_id not in seen_interfaces and interface.num_sites < num_sites_limit:
                dic = {'substrate':sub_formula, 'material_id': m[5],
                       'film':film_formula, 
                       'hkl_sub':match[0], 'hkl_film':match[1], 
                       'misfit_x': match[2][0], 'misfit_y': match[2][1], 
                       'misfit_x_sub': match[4][0], 'misfit_y_sub': match[4][1], 
                       'von_mises':round(match[3],3), 'termination': termination, 'n_at': interface.num_sites, 
                       'slab': None, 'substrate_density': None, 'film_density': None, 
                       'substrate_charge_density': None, 'film_charge_density': None,
                       'abs_charge_density':None}
                
                
                if interface.num_sites < num_sites_limit:
                    i += 1
                    
                    all_interfaces.append(interface)
                    seen_interfaces.add(interface_id)
                    
                    substrate_density = compute_surface_density(interface.substrate, select="top", layer_thickness = surface_thickness)
                    film_density = compute_surface_density(interface.film, select="bottom", layer_thickness = surface_thickness)

                    substrate_charge_density = compute_surface_charge_density(interface.substrate, select="top", layer_thickness = surface_thickness)
                    film_charge_density = compute_surface_charge_density(interface.film, select="bottom", layer_thickness = surface_thickness)

                    total_charge_density = abs(substrate_charge_density + film_charge_density)



                    if substrate_density > density_limit and film_density > density_limit and total_charge_density < charge_limit:
                        
                        dic['substrate_density'] = round(substrate_density,3)
                        dic['film_density'] = round(film_density,3)
                        dic['substrate_charge_density'] = round(substrate_charge_density,3)
                        dic['film_charge_density'] = round(film_charge_density,3)
                        dic['abs_charge_density'] = round(total_charge_density,3)

                        t1 = termination[0].replace('/', '')
                        t2 = termination[1].replace('/', '')
                        filename = f'{sub_formula}_{film_formula}_{"".join(map(str, substrate_miller))}_{"".join(map(str, film_miller))}_{interface.num_sites}at_{t1}_{t2}'
                        dic['slab'] = filename
                        import os
                        os.makedirs(f'{folder_name}/{sub_formula}/', exist_ok=True)
                        interface.to(filename=f'{folder_name}/{sub_formula}/{filename}.POSCAR', fmt="poscar")
                        interface.substrate.to(filename=f'{folder_name}/{sub_formula}/{filename}_substrate.POSCAR', fmt="poscar")
                        interface.film.to(filename=f'{folder_name}/{sub_formula}/{filename}_film.POSCAR', fmt="poscar")
                        dic_list.append(dic)
                        print('--- Interface is suitable. Writing...', filename)
                    else:
                        print(f'Density or charge condition has not passed:\nfilm_density = {film_density:.4f}\nsubst_density = {substrate_density:.4f}\ntotal_charge_density = {total_charge_density:.4f}')
                else:
                    print(f'N_at condition has not passed\nN_at = {interface.num_sites}')
            else:
                    # print('Same termination')
                    ''

    return dic_list, all_interfaces

## Interface with Li

In [20]:
#Necessary parameters

film_bulk = Structure.from_file("Li.cif")
film_bulk.add_oxidation_state_by_element({'Li': +1})


substrate_millers = [(0,0,1),(1,1,0), (1,1,1)]  # Specify the Miller index of the substrate
film_max_miller = 1  # Max Miller index for the film
num_sites_limit = 300  # Limit for total atoms in the interface
misfit = 5.1  # Max misfit percentage allowed
density_limit = 0.01
charge_limit = 0.2
film_thickness=5
film_thickness=13
substrate_thickness=10
# substrate_thickness=14
surface_thickness = 0.8


final_interfaces = []

film_miller_list = [(0, 0, 1), (1, 1, 0), (1, 1, 1)]  # Example set of Miller indices for the film

# for mp in dic_st_final.keys():
# # for mp in ['mp-13944']:
for mp in ['mp-755225']:
    substrate_bulk = dic_st_final[mp].add_oxidation_state_by_guess()
    print(mp, substrate_bulk.composition.reduced_formula)
    substrate = substrate_bulk.composition.reduced_formula
    film = film_bulk.composition.reduced_formula

    for substrate_miller in substrate_millers:
        print(substrate_miller)
        matches_i = matches(substrate_bulk, film_bulk, substrate_miller, film_max_miller =film_max_miller, misfit = misfit)
        for m in matches_i:
            film_miller = m[1]
            print(film_miller)
            m.append(mp)
            print(m)
            dic_list, interfaces = create_interfaces(
                    substrate_bulk, film_bulk, substrate_miller, film_miller, film_max_miller=film_max_miller,
                    num_sites_limit = num_sites_limit, density_limit = density_limit, charge_limit = charge_limit,
                    film_thickness=film_thickness, substrate_thickness=substrate_thickness, surface_thickness = 0.8, 
                    misfit = misfit, match = m, vacuum_over_film = 2,
                    folder_name = 'interfaces_with_Li_2026'
                )


            final_interfaces.extend(dic_list)


mp-755225 Li8ZrO6
(0, 0, 1)
Miller s/f: (0, 0, 1)/(1, 0, 0) Misfit: 2.16,9.00
Miller s/f: (0, 0, 1)/(1, 0, 0) Misfit: 6.07,9.00
Miller s/f: (0, 0, 1)/(1, 0, 0) Misfit: 2.16,19.00
Miller s/f: (0, 0, 1)/(1, 0, 0) Misfit: 2.16,19.00
Miller s/f: (0, 0, 1)/(1, 0, 0) Misfit: 5.08,19.00
Miller s/f: (0, 0, 1)/(1, 0, 0) Misfit: 5.08,19.00
Miller s/f: (0, 0, 1)/(1, 0, 0) Misfit: 6.07,19.00
Miller s/f: (0, 0, 1)/(1, 0, 0) Misfit: 6.07,19.00
Miller s/f: (0, 0, 1)/(1, 0, 0) Misfit: 12.04,19.00
Miller s/f: (0, 0, 1)/(1, 0, 0) Misfit: 12.04,19.00
Miller s/f: (0, 0, 1)/(1, 0, 0) Misfit: 13.04,19.00
Miller s/f: (0, 0, 1)/(1, 0, 0) Misfit: 13.04,19.00
Miller s/f: (0, 0, 1)/(1, 0, 0) Misfit: 16.03,19.00
Miller s/f: (0, 0, 1)/(1, 0, 0) Misfit: 16.03,19.00
Miller s/f: (0, 0, 1)/(1, 0, 0) Misfit: 2.61,9.00
Miller s/f: (0, 0, 1)/(1, 0, 0) Misfit: 2.61,9.00
Miller s/f: (0, 0, 1)/(1, 0, 0) Misfit: 6.28,9.00
Miller s/f: (0, 0, 1)/(1, 0, 0) Misfit: 6.28,9.00
Miller s/f: (0, 0, 1)/(1, 0, 0) Misfit: 7.06,25.00
Mil

In [20]:
dic_st_final['mp-13944']

In [23]:
import pandas as pd
df = pd.DataFrame(final_interfaces)
print(df[['termination', 'n_at', 'slab', 'substrate_density', 'film_density', 'substrate_charge_density', 'film_charge_density']])
df.to_csv("interface_Li_Li8ZrO6.csv")

                 termination  n_at  \
0     (Li_R-3m_1, Zr_Fmmm_1)   191   
1    (Li_R-3m_1, Li2O2_Cm_2)   191   
2     (Li_R-3m_1, O2_Fmmm_1)   191   
3   (Li_P4/mmm_1, Zr_P2/m_1)   268   
4   (Li_P4/mmm_1, LiO2_P1_3)   268   
5   (Li_P4/mmm_1, Li_P2/m_1)   268   
6     (Li_Cmmm_1, Zr_P2/m_1)    69   
7     (Li_Cmmm_1, LiO2_P1_3)    69   
8       (Li_Cmmm_1, O2_R3_3)   150   
9       (Li_Cmmm_1, Li_R3_3)   150   
10      (Li_R-3m_1, O2_R3_3)   146   
11      (Li_R-3m_1, Li_R3_3)   146   

                                             slab  substrate_density  \
0    Li8ZrO6_Li_001_111_191at_Li_R-3m_1_Zr_Fmmm_1              0.103   
1   Li8ZrO6_Li_001_111_191at_Li_R-3m_1_Li2O2_Cm_2              0.137   
2    Li8ZrO6_Li_001_111_191at_Li_R-3m_1_O2_Fmmm_1              0.137   
3    Li8ZrO6_Li_110_100_268at_Li_P4mmm_1_Zr_P2m_1              0.122   
4   Li8ZrO6_Li_110_100_268at_Li_P4mmm_1_LiO2_P1_3              0.122   
5    Li8ZrO6_Li_110_100_268at_Li_P4mmm_1_Li_P2m_1              0.091   
6

In [22]:
df


NameError: name 'df' is not defined

In [21]:
dic_st_final.keys()

dict_keys(['mp-755225', 'mp-542435', 'mp-3887', 'mp-13944', 'mp-23703', 'mp-2530', 'mp-1189725', 'mp-570935', 'mp-569782'])

## Interfaces with LGPS/LLZO



In [8]:
substrate_bulk = Structure.from_file("llzo/Li7La3Zr2O12.POSCAR")
substrate_bulk

In [39]:
#Necessary parameters

# substrate_bulk = st


substrate_bulk = Structure.from_file("LGPS.POSCAR")
substrate_bulk = Structure.from_file("llzo/Li7La3Zr2O12.POSCAR")
substrate_bulk.add_oxidation_state_by_guess()



substrate_millers = [(1,0,0),(0,0,1),(1,1,0), (1,1,1)]  # Specify the Miller index of the substrate
substrate_millers = [(0,0,1)]  # Specify the Miller index of the substrate
film_max_miller = 2  # Max Miller index for the film
num_sites_limit = 600  # Limit for total atoms in the interface
misfit = 10  # Max misfit percentage allowed
# density_limit = 0.05
density_limit = 0.05
charge_limit = 0.1
film_thickness=10
# film_thickness=13
substrate_thickness=10
# substrate_thickness=14
surface_thickness = 0.8


final_interfaces = []

film_miller_list = [(0, 0, 1), (1, 1, 0), (1, 1, 1)]  # Example set of Miller indices for the film

# for mp in dic_st_final.keys():
for mp in ['mp-13944',]:
    film_bulk = dic_st_final[mp].add_oxidation_state_by_guess()
    print(mp, film_bulk.composition.reduced_formula)
    substrate = substrate_bulk.composition.reduced_formula
    film = film_bulk.composition.reduced_formula

    for substrate_miller in substrate_millers:
        print(substrate_miller)
        matches_i = matches(substrate_bulk, film_bulk, substrate_miller, film_max_miller =film_max_miller, misfit = misfit)
        for m in matches_i:
            film_miller = m[1]
            print(film_miller)
            m.append(mp)
            print(m)
            dic_list, interfaces = create_interfaces(
                    substrate_bulk, film_bulk, substrate_miller, film_miller, film_max_miller=film_max_miller,
                    num_sites_limit = num_sites_limit, density_limit = density_limit, charge_limit = charge_limit,
                    film_thickness=film_thickness, substrate_thickness=substrate_thickness, surface_thickness = 0.8, 
                    misfit = misfit, match = m,
                    # folder_name = 'interfaces_with_LGPS'
                    folder_name = 'interfaces_with_LLZO'
                )


            final_interfaces.extend(dic_list)


mp-13944 Li3AlN2
(0, 0, 1)
Miller s/f: (0, 0, 1)/(1, 1, -1) Misfit: 0.15,1.00
ok
Miller s/f: (0, 0, 1)/(1, 1, -1)
Match area: 177.4897
Von_mises_strain: 0.0043
Film Misfit along x: 0.2
Film Misfit along y: 1.0
Sub Misfit along x: 0.0
Sub Misfit along y: 0.0


Miller s/f: (0, 0, 1)/(1, 1, -1) Misfit: 0.15,3.00
ok
Miller s/f: (0, 0, 1)/(1, 1, -1) Misfit: 0.15,3.00
ok
Miller s/f: (0, 0, 1)/(1, 1, -1) Misfit: 1.83,3.00
ok
Miller s/f: (0, 0, 1)/(1, 1, -1) Misfit: 1.83,3.00
ok
Miller s/f: (0, 0, 1)/(1, 1, -1) Misfit: 1.00,1.00
ok
(1, 1, -1)
[(0, 0, 1), (1, 1, -1), ['0.2', '1.0'], np.float64(0.004312205007286092), ['0.0', '0.0'], 'mp-13944']
[('N2_Pmma_8', 'Li3La_P4/mbm_8'), ('N2_Pmma_8', 'O2_Pbam_4'), ('N2_Pmma_8', 'LiO2_Pba2_12'), ('Li2Al_Pba2_12', 'Li3La_P4/mbm_8'), ('Li2Al_Pba2_12', 'O2_Pbam_4'), ('Li2Al_Pba2_12', 'LiO2_Pba2_12'), ('Li_Pbam_4', 'Li3La_P4/mbm_8'), ('Li_Pbam_4', 'O2_Pbam_4'), ('Li_Pbam_4', 'LiO2_Pba2_12'), ('N2_P4/mmm_2', 'Li3La_P4/mbm_8'), ('N2_P4/mmm_2', 'O2_Pbam_4'), ('N

In [10]:
from pymatgen.core.structure import Structure
from pymatgen.core.surface import SlabGenerator, generate_all_slabs

slabgen = SlabGenerator(initial_structure=substrate_bulk,
                        miller_index=(0,0,1),
                        min_slab_size=10.0,
                        min_vacuum_size=15.0,
                        center_slab=True,
                       )

slabs = slabgen.get_slabs()

In [15]:
print(dir(slabgen))
print(len(slabs))
slabs[0]



['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_normal', '_proj_height', 'center_slab', 'get_slab', 'get_slabs', 'in_unit_planes', 'lll_reduce', 'max_normal_search', 'miller_index', 'min_slab_size', 'min_vac_size', 'move_to_other_side', 'nonstoichiometric_symmetrized_slab', 'oriented_unit_cell', 'parent', 'primitive', 'reorient_lattice', 'repair_broken_bonds', 'slab_scale_factor']
4


/home/anton/python3.10_env/notebookenv_3.10/lib/python3.10/site-packages/pymatgen/analysis/local_env.py:4019: UserWarning:

No oxidation states specified on sites! For better results, set the site oxidation states in the structure.

/home/anton/python3.10_env/notebookenv_3.10/lib/python3.10/site-packages/pymatgen/analysis/local_env.py:4021: UserWarning:

No oxidation states specified on sites! For better results, set the site oxidation states in the structure.

/home/anton/python3.10_env/notebookenv_3.10/lib/python3.10/site-packages/pymatgen/analysis/local_env.py:3933: UserWarning:

CrystalNN: cannot locate an appropriate radius, covalent or atomic radii will be used, this can lead to non-optimal results.

/home/anton/python3.10_env/notebookenv_3.10/lib/python3.10/site-packages/pymatgen/analysis/local_env.py:4019: UserWarning:

No oxidation states specified on sites! For better results, set the site oxidation states in the structure.

/home/anton/python3.10_env/notebookenv_3.10/lib/pyt

In [37]:
print(dir(slabs[0]))
slabs[3].is_symmetric()
slab = slabgen.nonstoichiometric_symmetrized_slab(slabs[0])

# if slab:
#     print("⚠️ Найден нестехиометрический симметризованный slab!")
#     # print("Формула:", slab.composition)
#     # slab.to(fmt="poscar", filename="POSCAR_LLZO_nonstoich.vasp")
    
# else:
#     print("Все симметричные slab'ы стехиометрические ✅")
# dir(slab)
print(slab)


['CellType', 'DISTANCE_TOLERANCE', 'REDIRECT', '__abstractmethods__', '__annotations__', '__class__', '__class_getitem__', '__contains__', '__delattr__', '__delitem__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__get_pydantic_core_schema__', '__get_pydantic_json_schema__', '__get_validators__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__iadd__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__modify_schema__', '__module__', '__mul__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__reversed__', '__rmul__', '__setattr__', '__setitem__', '__sizeof__', '__slotnames__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_calculate', '_charge', '_generic_json_schema', '_get_neighbor_list_py', '_get_sites_to_draw', '_ipython_display_', '_lattice', '_prep_calculator', '_properties', '_relax', '_repr_mimebundle_', '_sites', '_validate_monty', 'add_adsorbate_atom', 'add_oxidation_

In [31]:
import pandas as pd
df = pd.DataFrame(final_interfaces)
print(df[['termination', 'n_at', 'slab', 'substrate_density', 'film_density', 'substrate_charge_density', 'film_charge_density']])
df.to_csv("interface_with_LLZO_summary2.csv")

KeyError: "None of [Index(['termination', 'n_at', 'slab', 'substrate_density', 'film_density',\n       'substrate_charge_density', 'film_charge_density'],\n      dtype='object')] are in the [columns]"

In [22]:
from pymatgen.analysis.interfaces.zsl import ZSLGenerator
# from pymatgen.analysis.interfaces.coherent_interfaces import generate_coherent_interface

# Your own substrate slab
st = dic_st_final['mp-3887']
# st = Structure.from_file('LiCoO2.cif')


slabgen = SlabGenerator(initial_structure = st, miller_index = (0,0,1), min_slab_size = 7, 
                            min_vacuum_size = 10, lll_reduce = True, center_slab = False, primitive = True)
slabs = slabgen.get_slabs(symmetrize=True )
print(len(slabs))
substrate_slab = slabs[1]

2
